## Loading PDF

In [1]:
!pip install --q unstructured langchain
!pip install --q "unstructured[all-docs]"


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\Nitro 5\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\Nitro 5\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [28]:
local_path = "D:/Super AI SS4\AI-builder/Large-Language-Models-(LLMs)/try-it/RAG/data/FSCI4_วท.บ.วิทยาการคอมพิวเตอร์ประยุกต์_2565.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [30]:
# Preview first page
print(data[0].page_content)

หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ประยุกต์ หลักสูตรปรับปรุง พ.ศ. 2565

ภาควิชาคณิตศาสตร์ คณะวิทยาศาสตร์ มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี

มคอ.2 KMUTT (ฉบับปรับปรุง ณ วันที

่ 19)

เม.ย. 64)

ชื

่อหลักสูตร

: หลักสูตรวิทยาศาสตรบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ประยุกต์ (ภาษาไทย) (ภาษาอังกฤษ) : Bachelor of Science Program in Applied Computer Science

ชื

่อปริญญาและสาขาวิชา

่อเต็ม (ภาษาไทย) ชื (ภาษาอังกฤษ) ่อย่อ (ภาษาไทย) ชื (ภาษาอังกฤษ)

: วิทยาศาสตรบัณฑิต (วิทยาการคอมพิวเตอร์ประยุกต์) : Bachelor of Science (Applied Computer Science) : วท.บ. (วิทยาการคอมพิวเตอร์ประยุกต์) : B.Sc. (Applied Computer Science)

โครงสร้างหลักสูตร

จำนวนหน่วยกิตรวมตลอดหลักสูตร โครงสร้างหลักสูตร

129

หน่วยกิต

ก. หมวดวิชาศึกษาทั ข. หมวดวิชาเฉพาะ - วิชาแกน - วิชาเฉพาะด้าน - วิชาเลือก

่วไป

ค. หมวดวิชาเลือกเสรี

31 92 24 47 21 6

หน่วยกิต หน่วยกิต หน่วยกิต หน่วยกิต หน่วยกิต หน่วยกิต

อนุมัติจากสภา มจธ.ครั

้งที

่ 268 (1 ธ.ค. 64)

2

เม.ย. 64)

มคอ.2 KMUTT (ฉบับปรับปรุง ณ วันที

่ 19)



## Vector Embeddings

In [31]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [32]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
nomic-embed-text:latest	0a109f422b47	274 MB	Less than a second ago	
mistral:latest         	2ae6f6dd7a3d	4.1 GB	46 hours ago          	
llama3:latest          	365c0bd3c000	4.7 GB	46 hours ago          	


In [8]:
!pip install --q chromadb
!pip install --q langchain-text-splitters


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\Nitro 5\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\Nitro 5\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [33]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [34]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [35]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 23/23 [00:55<00:00,  2.40s/it]


## Retrieval

In [36]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [37]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [38]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [39]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [40]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]


'This document appears to be a report on the "Global Cooperation Barometer" by McKinsey & Company, which tracks global cooperation and trends from 2012 to 2022. The report covers five pillars of global cooperation: trade and capital, innovation and technology, climate and natural capital, health and wellness, and peace and security. It provides data and trendlines on various metrics related to these pillars, including economic flows, intellectual property, migration, mortality rates, and more.'

In [42]:
chain.invoke("เป็นหลักสูตรของมหาวิทยาลัยไหน")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


'Based on the provided context, it appears to be a course from King Mongkut University of Technology Thonburi (KMUTT). The metadata indicates that this is a PDF document containing information about FSCI4 วท.บ. วิทยาการคอมพิวเตอร์ประยุกต์ _2565, which is likely a course in the field of Computer Science and Information Technology at KMUTT.'

In [26]:
# Delete all collections in the db
vector_db.delete_collection()